In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline

import jieba
import re
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn import metrics
import random

import fasttext

In [2]:
data_path = '../input/train_first.csv'
df = pd.read_csv(data_path,header = 0, encoding='utf-8')

test_data_path = '../input/predict_first.csv'
test_df = pd.read_csv(test_data_path,header = 0, encoding='utf-8')

In [3]:
stop_word = []
stop_words_path = '../input/stop_word.txt'
with open(stop_words_path,encoding='utf-8') as f:
    for line in f.readlines():
        stop_word.append(line.strip())
stop_word.append(' ')

def clean_str(stri):
    stri = re.sub(r'[a-zA-Z0-9]+','',stri)
    cut_str = jieba.cut(stri.strip())
    list_str = [word for word in cut_str if word not in stop_word]
    stri = ' '.join(list_str)
    return stri

In [4]:
df['Discuss'] = df['Discuss'].map(lambda x : clean_str(x))
test_df['Discuss'] = test_df['Discuss'].map(lambda x : clean_str(x))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.847 seconds.
Prefix dict has been built succesfully.


In [5]:
def fillnull(x):
    if x == '':
        return '空白'
    else:
        return x

In [6]:
df['Discuss'] = df['Discuss'].map(lambda x: fillnull(x))
test_df['Discuss'] = test_df['Discuss'].map(lambda x: fillnull(x))

In [7]:
df.head()

,Id,Discuss,Score
0,201e8bf2-77a2-3a98-9fcf-4ce03914e712,好大 一个 游乐 公园 已经 次 感觉 玩够 第三 第四次,5
1,f4d51947-eac4-3005-9d3c-2f32d6068a2d,新 中国 成立 举行 中国 人 来说 重要 深刻 意义,4
2,74aa7ae4-03a4-394c-bee0-5702d3a3082a,庐山 瀑布 有名 多个 瀑布 最 好看 非 三叠 泉莫属 推荐 一去,4
3,099661c2-4360-3c49-a2fe-8c783764f7db,觉得 颐和园 北京 最值 一起 地方 相比 下 门票 最贵 故宫 雄伟 气势磅礴 颐和园 宁...,5
4,97ca672d-e558-3542-ba7b-ee719bba1bab,迪斯尼 一日游,5


In [8]:
def fasttext_data(data,label):
    fasttext_data = []
    for i in range(len(label)):
        sent = data[i]+"\t__label__"+str(int(label[i]))
        fasttext_data.append(sent)
    with open('../input/train.txt','w', encoding = 'utf-8-sig') as f:
        for data in fasttext_data:
            f.write(data)
            f.write('\n')
    return '../input/train.txt'

def get_predict(pred):
    score = np.array([1,2,3,4,5])
    pred2 = []
    for p in pred:
        pr = np.sum(p * score)
        pred2.append(pr)
    return np.array(pred2)

def rmsel(true_label,pred):
    rmse = np.sqrt(metrics.mean_squared_error(true_label, pred))
    return rmse / (1 + rmse)

In [ ]:
def fast_cv(df):
    X = df['Discuss'].values
    y = df['Score'].values
    fast_pred = []
    folds = list(KFold(n_splits=5, shuffle=True, random_state=2018).split(X, y))
    for train_index, test_index in folds:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        train_file = fasttext_data(X_train,y_train)
        classifier = fasttext.supervised(train_file,'model',lr=0.01,dim=128,label_prefix="__label__", encoding = 'utf-8-sig')
        
        result = classifier.predict_proba(df.loc[test_index,'Discuss'].tolist(),k=5)
        print(result[0:100])
        pred = [[int(sco) * proba for sco,proba in result_i] for result_i in result]
        pred = [sum(pred_i) for pred_i in pred]
        print(pred[0:100])
        print(rmsel(y_test,pred))
        
        test_result = classifier.predict_proba(test_df['Discuss'].tolist(),k=5)
        fast_predi = [[int(sco) * proba for sco,proba in result_i] for result_i in test_result]
        fast_predi = [sum(pred_i) for pred_i in fast_predi]
        fast_pred.append(fast_predi)
    
    fast_pred = np.array(fast_pred)
    fast_pred = np.mean(fast_pred, axis=0)
    return fast_pred

In [ ]:
test_pred1 = fast_cv(df)
print(test_pred1[0:100])

In [ ]:
import fasttext
classifier = fasttext.load_model('../input/model.bin', label_prefix='__label__', encoding = 'gbk')
result = classifier.predict_proba(['这里实在是太差了'], k=5)
print(result)

[[]]
